In [543]:
import sys 
import _pickle as pickle
sys.path.append("/work/corpus_reader")

In [544]:
sys.path.append("/work/corpus_reader/corpus_reader/miwa_reader")
from miwa_reader import MiwaReader

In [545]:
reader = MiwaReader()

In [546]:
docs = reader.read("/work/miwa2016/corpus/train/")

In [547]:
path_file = "testing_data/train_pathsv3"
relation_file = "testing_data/train_relationsv3.txt"

In [548]:
# find the lowest common ancestor of two entity token
def find_lowest_ancest(token_idx, check, sen, word, dep, pos_tag, pos):
    '''
    '''
    if check[token_idx] == 0:
        check[token_idx] = 1
        word[-1].append(sen.tokens[token_idx].text)
        dep[-1].append(sen.tokens[token_idx].dep_type)
        pos_tag[-1].append(sen.tokens[token_idx].pos)
        pos[-1].append(token_idx + 1)
        if sen.tokens[token_idx].dep_type != "ROOT":
            for tok_idx in range(len(sen.tokens)):
                if sen.tokens[tok_idx].id == sen.tokens[token_idx].dep_head:
                    token_idx = find_lowest_ancest(tok_idx, check, sen, word, dep, pos_tag, pos)
                    break
    return token_idx

In [549]:
# declare variable
words_seq = []
deps_seq = []
pos_tags_seq = []
word_path1 = []
word_path2 = []
dep_path1 = []
dep_path2 = []
pos_tags_path1 = []
pos_tags_path2 = []
pos_path1 = []
pos_path2 = []
childs_path1 = []
childs_path2 = []
relation = []
temp = 0

In [550]:
# obtain our input data for miwa dataset, there will be C(#entity mention, 2) of input data for every sentence in each file
for file_name in docs:
    for sentence_index in range(len(docs[file_name].sentences)):
        # skip those sentences sequences larger than 90 or less than 1
        if len(docs[file_name].sentences[sentence_index].tokens) >= 90 or len(docs[file_name].sentences[sentence_index].tokens) <= 1:
            continue
        for entity1 in range(len(docs[file_name].sentences[sentence_index].entity_mentions)):
            for entity2 in range(entity1 + 1, len(docs[file_name].sentences[sentence_index].entity_mentions)):
                
                # get words_seq, deps_seq, pos_tags_seq of each sentence 
                words_seq.append([])
                deps_seq.append([])
                pos_tags_seq.append([])
                
                for token_index in range(len(docs[file_name].sentences[sentence_index].tokens)):
                    words_seq[-1].append(docs[file_name].sentences[sentence_index].tokens[token_index].text)
                    deps_seq[-1].append(docs[file_name].sentences[sentence_index].tokens[token_index].dep_type)
                    pos_tags_seq[-1].append(docs[file_name].sentences[sentence_index].tokens[token_index].pos)
                    
                # get word_path, dep_path, pos_tag_path, pos_path of each sentence 
                word_path1.append([])
                word_path2.append([])
                dep_path1.append([])
                dep_path2.append([])
                pos_tags_path1.append([])
                pos_tags_path2.append([])
                pos_path1.append([])
                pos_path2.append([])
                check_list = [0 for i in range(len(docs[file_name].sentences[sentence_index].tokens))]
                
                # generate ralation file
                break_sig = 0
                for rlt_idx in range(len(docs[file_name].sentences[sentence_index].relation_mentions)):
                    if (docs[file_name].sentences[sentence_index].relation_mentions[rlt_idx].arg1.string == docs[file_name].sentences[sentence_index].entity_mentions[entity1].string and  docs[file_name].sentences[sentence_index].relation_mentions[rlt_idx].arg2.string == docs[file_name].sentences[sentence_index].entity_mentions[entity2].string ) or (docs[file_name].sentences[sentence_index].relation_mentions[rlt_idx].arg1.string == docs[file_name].sentences[sentence_index].entity_mentions[entity2].string and docs[file_name].sentences[sentence_index].relation_mentions[rlt_idx].arg2.string == docs[file_name].sentences[sentence_index].entity_mentions[entity1].string):
                        relation.append(docs[file_name].sentences[sentence_index].relation_mentions[rlt_idx].type)
                        break_sig = 1
                        break
                if break_sig == 0:
                    relation.append("NONE")
                
                # find the lowest common ancestor
                for tok_idx in range(len(docs[file_name].sentences[sentence_index].tokens)):
                    if str(docs[file_name].sentences[sentence_index].entity_mentions[entity1].char_e) == docs[file_name].sentences[sentence_index].tokens[tok_idx].char_e:
                        top = find_lowest_ancest(tok_idx, check_list, docs[file_name].sentences[sentence_index], word_path1, dep_path1, pos_tags_path1, pos_path1)
                        break

                for tok_idx in range(len(docs[file_name].sentences[sentence_index].tokens)):                
                    if str(docs[file_name].sentences[sentence_index].entity_mentions[entity2].char_e) == docs[file_name].sentences[sentence_index].tokens[tok_idx].char_e:
                        top = find_lowest_ancest(tok_idx, check_list, docs[file_name].sentences[sentence_index], word_path2, dep_path2, pos_tags_path2, pos_path2)
                        while top != pos_path1[-1][-1] - 1:
                            word_path1[-1].pop()
                            dep_path1[-1].pop()
                            pos_tags_path1[-1].pop()
                            pos_path1[-1].pop()
                                                   
                check_list.clear()
                
                # restrict our LCA path in dependency layer less than 20
                if len(pos_path1[-1]) > 20 or len(pos_path2[-1]) > 20:
                    words_seq.pop()
                    deps_seq.pop()
                    pos_tags_seq.pop()
                    word_path1.pop()
                    word_path2.pop()
                    dep_path1.pop()
                    dep_path2.pop()
                    pos_tags_path1.pop()
                    pos_tags_path2.pop()
                    pos_path1.pop()
                    pos_path2.pop()
                    relation.pop()
                    continue
                
                # get childs_path of each sentence 
                childs_path1.append([])
                childs_path2.append([])

                for tok_targ in pos_path1[-1]:
                    childs_path1[-1].append([])
                    for tok_idx in range(len(docs[file_name].sentences[sentence_index].tokens)):
                        if docs[file_name].sentences[sentence_index].tokens[tok_idx].dep_head == docs[file_name].sentences[sentence_index].tokens[tok_targ - 1].id:
                            childs_path1[-1][-1].append(tok_idx + 1)
                
                for tok_targ in pos_path2[-1]:
                    childs_path2[-1].append([])
                    for tok_idx in range(len(docs[file_name].sentences[sentence_index].tokens)):
                        if docs[file_name].sentences[sentence_index].tokens[tok_idx].dep_head == docs[file_name].sentences[sentence_index].tokens[tok_targ - 1].id:
                            childs_path2[-1][-1].append(tok_idx + 1)
                
                # restrict our children number of each entity less than 20
                for i in childs_path1[-1]:
                    if len(i) > 20:
                        words_seq.pop()
                        deps_seq.pop()
                        pos_tags_seq.pop()
                        word_path1.pop()
                        word_path2.pop()
                        dep_path1.pop()
                        dep_path2.pop()
                        pos_tags_path1.pop()
                        pos_tags_path2.pop()
                        pos_path1.pop()
                        pos_path2.pop()
                        childs_path1.pop()
                        childs_path2.pop()
                        relation.pop()
                        continue
                for i in childs_path2[-1]:
                    if len(i) > 20:
                        words_seq.pop()
                        deps_seq.pop()
                        pos_tags_seq.pop()
                        word_path1.pop()
                        word_path2.pop()
                        dep_path1.pop()
                        dep_path2.pop()
                        pos_tags_path1.pop()
                        pos_tags_path2.pop()
                        pos_path1.pop()
                        pos_path2.pop()
                        childs_path1.pop()
                        childs_path2.pop()
                        relation.pop()
                        continue

In [551]:
# make data the multiple of our batch_size
while len(words_seq) % 10 != 0:
    words_seq.pop()
    deps_seq.pop()
    pos_tags_seq.pop()
    word_path1.pop()
    word_path2.pop()
    dep_path1.pop()
    dep_path2.pop()
    pos_tags_path1.pop()
    pos_tags_path2.pop()
    pos_path1.pop()
    pos_path2.pop()
    childs_path1.pop()
    childs_path2.pop()
    relation.pop()
print(len(words_seq))

76390


In [552]:
# create our file 
# if it is training then "train_pathsv3", testing then "test_pathsv3"

with open(path_file, "wb") as f:
    pickle.dump((words_seq, deps_seq, pos_tags_seq, word_path1, word_path2, dep_path1, dep_path2, pos_tags_path1, pos_tags_path2, pos_path1, pos_path2, childs_path1, childs_path2), f)
    f.close()
    
with open(relation_file, "w") as f:
    for i in range(len(relation)):
        f.write("{} {}\n".format(i + 1, relation[i]))
    f.close()